# In this notebook, I tried to implement and explain the implementation of a binary classification deep neural network.

In [1]:
import pandas as pd
import numpy as np

In [3]:
# Get the initial state of the neural network (default values of weights and biases):

def initialize(dimensions): # Dimensions is an array containing the desired lengths of each layer
  parameters = {}
  layers = len(dimensions) # Number of layers

  for layer in range(1, layers):
    parameters["W" + str(layer)] = np.random.randn(dimensions[layer], dimensions[layer-1]) # Matrix of weights of the layer "layer" 's neurons
    parameters["b" + str(layer)] = np.random.randn(dimensions[layer], 1) # Matrix of biases of the layer "layer" 's neurons
  
  return parameters

In [4]:
# Forward Propagation:

def forward_propagation(X, parameters):

  activations = { "A0" : X }
  layers = len(parameters) // 2 # Since the parameters dictionary contains both the Matrix of weights and array of biases for each layer
  
  for layer in range(1, layers + 10):
    Z = parameters["W" + str(layer)].dot(activations["A" + str(layer - 1)]) + parameters["b" + str(layer)]
    activations["A" + str(layer)] = 1 / (1 + np.exp(-Z))

  return activations

In [5]:
# Back Propagation

def back_propagation(y, activations, parameters):
  m = y.shape[1] # Number of "y"s we have
  layers = len(parameters) // 2 # number of layers

  dZ = activations["A" + str(layers)] - y # We start from the last layer's activation

  gradients = {} # "dW"s and "db"s

  for layer in reversed(range(1, layers+ 1)):
    gradients["dW" + str(layer)] = 1/m * np.dot(dZ, activations["A" + str(layer - 1)].T)
    gradients["db" + str(layer)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    if layer > 1: # Because it doesn't make sense calculating dZ1 to be used by dW0 and db0 which don't exist
      dZ = np.dot(parameters["W" + str(layer)].T, dZ) * activations["A" + str(layer - 1)] * (1 - activations["A" + str(layer - 1)])

  return gradients

In [6]:
# Update weights and  biases:

def update(parameters, gradients, lr): # lr is the learning rate
  layers = len(parameters) // 2
  for layer in range(1, layers + 1):
    parameters["W" + str(layer)] = parameters["W" + str(layer)] - lr * gradients["dW" + str(layer)]
    parameters["b" + str(layer)] = parameters["b" + str(layer)] - lr * gradients["db" + str(layer)]

  return parameters

In [7]:
# Log loss function to keep track of the loss:

def log_loss(y, A):
  m = y.shape[1]
  return 1/m * np.sum(- y * np.log(A) - (1 - y) * np.log(1-A))

In [8]:
# Predict function:

def predict(X, parameters):
  activations = forward_propagation(X, parameters)
  Af = activations["A" + str(len(parameters) // 2)]
  return Af > 1

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def neural_network(X, y, dimensions, lr, n_iter):

  parameters = initialize(dimensions)
  layers = len(parameters) // 2

  train_loss = []
  train_acc = []
  train_recall = []
  train_precision = []
  train_fscores = []

  for epoch in range(n_iter):
    activations = forward_propagation(X, parameters)
    gradients = back_propagation(y, activations, parameters)
    parameters = update(parameters, gradients, lr)

    Af = activations["A" + str(layers)]
    y_pred = predict(X, parameters)

    train_loss.append(log_loss(y, Af))
    train_acc.append(accuracy_score(y, y_pred))
    train_recall.appen(recall_score(y, y_pred))
    train_precision.appen(precision_score(y, y_pred))
    train_fscores.appen(f1_score(y, y_pred))

  return (parameters, train_loss, train_acc, train_recall, train_precision, train_fscores)

In [11]:
# Define your X, y dimensions, learning rate and number of epochs here:

# Call the function neural_network

In [12]:
import matplotlib.pyplot as plt

def plot_metrics(train_loss, train_acc, train_recall, train_precision, train_fscores):
  fig = plt.figure(figsize = (10,6))
  plt.plot(train_loss)
  plt.title('model Loss')
  plt.ylabel('Loss')
  plt.xlabel('epoch')
  plt.show()

  plt.plot(train_acc)
  plt.title('model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('epoch')
  plt.show()

  plt.plot(train_recall)
  plt.title('model recall')
  plt.ylabel('Recall')
  plt.xlabel('epoch')
  plt.show()

  plt.plot(train_precision)
  plt.title('model precision')
  plt.ylabel('Precision')
  plt.xlabel('epoch')
  plt.show()

  plt.plot(train_fscores)
  plt.title('model F-score')
  plt.ylabel('F-Score')
  plt.xlabel('epoch')
  plt.show()